## Import modules

In [1]:
import cobra
from tqdm import tqdm
import numpy as np
import math

## Model

In [2]:
model = cobra.io.read_sbml_model('Models/legacy/Mtu_iNJ661.xml')
model

Name,model
Memory address,0x010e428780
Number of metabolites,826
Number of reactions,1028
Objective expression,-1.0*biomass_Mtb_9_60atp_reverse_b7278 + 1.0*biomass_Mtb_9_60atp
Compartments,[


## Elimination list and model config

In [3]:
# elilist = [model.reactions.index(rxn.id) for rxn in model.reactions if model.reactions.get_by_id(rxn.id).gene_reaction_rule == '']
model.solver = 'glpk'

## Core model optimization

In [4]:
solWT = model.optimize()
solWT

,fluxes,reduced_costs
2AGPEAT160,0.000000,-4.671093e-02
2AGPEAT180,0.000000,-4.671093e-02
4HBAt,0.000000,0.000000e+00
4HTHRS,0.000000,-7.201269e-02
5DGLCNR,0.000000,-0.000000e+00
...,...,...
XPPT,0.000000,2.602085e-18
XYLK,0.000000,0.000000e+00
XYLabc,0.000000,-2.335547e-02
YUMPS,0.000000,-6.938894e-18


## Objective value and fluxes

In [5]:
grWT = solWT.objective_value
J = solWT.fluxes

## Flux operations to obtain the genes of the reactions having non-zero flux

In [8]:
Jnz_rxns_idx = np.flatnonzero(J)
Jnz_rxns_genes_frozenset = {model.reactions[rxn_idx].genes for rxn_idx in Jnz_rxns_idx if model.reactions[rxn_idx].gene_reaction_rule != ''}
Jnz_rxns_genes_idx = np.unique(np.array([model.genes.index(genes) for single_frozenset in Jnz_rxns_genes_frozenset for genes in single_frozenset]))
Jnz_rxns_genes_idx

array([  2,   5,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,
        19,  20,  21,  22,  34,  42,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  54,  55,  56,  57,  60,  61,  62,  63,  66,  68,  81,
        82,  83,  88,  92,  94,  98,  99, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 119, 121, 122, 127, 128, 129, 131, 132,
       148, 158, 159, 160, 161, 163, 164, 165, 166, 168, 174, 175, 176,
       178, 179, 180, 181, 183, 189, 190, 191, 192, 193, 194, 197, 198,
       202, 203, 204, 209, 210, 211, 212, 214, 215, 216, 217, 218, 219,
       220, 224, 225, 226, 227, 228, 229, 230, 231, 232, 241, 249, 250,
       251, 252, 253, 254, 259, 261, 263, 264, 265, 266, 268, 269, 270,
       271, 272, 273, 276, 277, 278, 279, 280, 282, 283, 284, 288, 289,
       290, 291, 293, 294, 295, 297, 298, 299, 300, 301, 304, 305, 306,
       307, 316, 318, 319, 320, 321, 322, 327, 335, 336, 340, 346, 349,
       350, 351, 354, 355, 357, 358, 359, 360, 362, 363, 364, 36

## Single lethal genes

In [13]:
Jsl_genes_idx = []
for delIdx_i in Jnz_rxns_genes_idx:
    with model:
        model.genes[delIdx_i].knock_out()
        solKO_i = model.slim_optimize()
        if solKO_i < 0.01 * grWT or math.isnan(solKO_i) == True:
            Jsl_genes_idx.append(int(delIdx_i))
Jsl_genes_idx = np.array(Jsl_genes_idx)
Jsl_genes_idx.shape

(188,)

## Eliminating single lethal genes to proceed further

In [10]:
Jnz_copy = np.setdiff1d(Jnz_rxns_genes_idx,Jsl_genes_idx)
Jnz_copy

array([  9,  10,  11,  15,  16,  17,  18,  20,  21,  42,  44,  54,  55,
        61,  62,  63,  82,  83,  88, 101, 102, 104, 105, 106, 107, 108,
       109, 110, 111, 131, 132, 158, 159, 160, 163, 164, 165, 166, 168,
       179, 180, 181, 194, 210, 211, 212, 214, 215, 216, 217, 218, 219,
       228, 232, 241, 249, 250, 251, 252, 253, 254, 261, 264, 269, 270,
       271, 272, 273, 282, 283, 284, 290, 291, 293, 294, 295, 301, 304,
       305, 306, 316, 318, 319, 320, 335, 336, 349, 350, 351, 354, 355,
       357, 358, 359, 388, 401, 402, 403, 412, 413, 426, 429, 431, 432,
       435, 436, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448,
       449, 450, 451, 464, 489, 508, 509, 510, 511, 512, 513, 514, 517,
       519, 520, 523, 524, 525, 537, 561, 562, 564, 566, 567, 579, 605,
       606, 607, 608, 609, 610, 614, 615, 616, 619, 625, 626, 629, 634,
       635, 646, 647, 648])

## Double lethal genes

### First part

In [11]:
Jdl_idx = []
for delIdx_i in tqdm(Jnz_copy,desc="Part 1 of 2"):
    with model:
        model.genes[delIdx_i].knock_out()
        solKO_i = model.optimize()
        newnnz = np.flatnonzero(solKO_i.fluxes)
        newnnz_rxns_genes_frozenset = {model.reactions[rxn_idx].genes for rxn_idx in newnnz if model.reactions[rxn_idx].gene_reaction_rule != ''}
        newnnz_rxns_genes_idx = np.unique(np.array([model.genes.index(genes) for single_frozenset in newnnz_rxns_genes_frozenset for genes in single_frozenset]))
        Jnz_i = np.setdiff1d(newnnz_rxns_genes_idx,Jnz_rxns_genes_idx)

        for delIdx_j in Jnz_i:
            with model:
                model.genes[delIdx_j].knock_out()
                solKO_ij = model.slim_optimize()
                if solKO_ij < 0.01 * grWT or math.isnan(solKO_ij) == True:
                    Jdl_idx.append([int(delIdx_i),int(delIdx_j)])

Part 1 of 2: 100%|██████████| 160/160 [00:07<00:00, 21.51it/s]


In [12]:
len(Jdl_idx)

15

### Second part

In [14]:
for delIdx_i in tqdm(Jnz_copy,desc="Part 2 of 2"):
    for delIdx_j in Jnz_copy:
        if np.where(Jnz_copy==delIdx_j) < np.where(Jnz_copy==delIdx_i):
            with model:
                model.genes[delIdx_i].knock_out()
                model.genes[delIdx_j].knock_out()
                solKO_ij = model.slim_optimize()
                if solKO_ij < 0.01 * grWT or math.isnan(solKO_ij) == True:
                    Jdl_idx.append([int(delIdx_i),int(delIdx_j)])

Part 2 of 2: 100%|██████████| 160/160 [00:26<00:00,  6.03it/s]


In [15]:
len(Jdl_idx)

49

In [16]:
Jdl_idx

[[88, 89],
 [88, 90],
 [168, 657],
 [273, 485],
 [273, 486],
 [293, 233],
 [294, 233],
 [295, 233],
 [301, 485],
 [301, 486],
 [426, 506],
 [566, 352],
 [579, 515],
 [619, 0],
 [619, 348],
 [10, 9],
 [55, 54],
 [83, 82],
 [102, 101],
 [211, 210],
 [216, 215],
 [282, 212],
 [284, 283],
 [291, 290],
 [293, 232],
 [294, 232],
 [295, 232],
 [306, 305],
 [320, 319],
 [336, 335],
 [402, 401],
 [413, 412],
 [436, 435],
 [512, 511],
 [513, 293],
 [513, 294],
 [513, 295],
 [514, 212],
 [517, 282],
 [517, 514],
 [520, 519],
 [562, 561],
 [607, 605],
 [607, 606],
 [608, 607],
 [609, 607],
 [610, 579],
 [626, 625],
 [635, 634]]

In [39]:
single = model.genes.get_by_any(Jsl_genes_idx.tolist())
single

[<Gene Rv3815c at 0x10e3efc18>,
 <Gene Rv1323 at 0x10e3efe10>,
 <Gene Rv3280 at 0x10e40b048>,
 <Gene Rv1302 at 0x10e40b2e8>,
 <Gene Rv1654 at 0x10e40b390>,
 <Gene Rv1653 at 0x10e40b438>,
 <Gene Rv3002c at 0x10e40b780>,
 <Gene Rv1655 at 0x10e40b978>,
 <Gene Rv0733 at 0x10ea63198>,
 <Gene Rv1285 at 0x10ea638d0>,
 <Gene Rv1286 at 0x10ea63978>,
 <Gene Rv0777 at 0x10ea63a20>,
 <Gene Rv0357c at 0x10ea63ac8>,
 <Gene Rv3793 at 0x10ea63b70>,
 <Gene Rv3794 at 0x10ea63c18>,
 <Gene Rv3795 at 0x10ea63cc0>,
 <Gene Rv3792 at 0x10ea63d68>,
 <Gene Rv1652 at 0x10ea74048>,
 <Gene Rv3248c at 0x10ea740f0>,
 <Gene Rv0957 at 0x10ea742e8>,
 <Gene Rv2981c at 0x10ea746d8>,
 <Gene Rv3423c at 0x10ea74828>,
 <Gene Rv2192c at 0x10ea8a0f0>,
 <Gene Rv3708c at 0x10ea8a828>,
 <Gene Rv2201 at 0x10ea8a978>,
 <Gene Rv1380 at 0x10ea8ac18>,
 <Gene Rv3709c at 0x10ea8acc0>,
 <Gene Rv2121c at 0x10ea8af60>,
 <Gene Rv1383 at 0x10e39ba20>,
 <Gene Rv1011 at 0x10e39bb70>,
 <Gene Rv2612c at 0x10e39bc18>,
 <Gene Rv1885c at 0x10e39bf6

In [40]:
[model.genes.index(gene) for gene in single]

[2,
 5,
 8,
 12,
 13,
 14,
 19,
 22,
 34,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 56,
 57,
 60,
 66,
 68,
 81,
 92,
 94,
 98,
 99,
 103,
 119,
 121,
 122,
 127,
 128,
 129,
 148,
 161,
 174,
 175,
 176,
 178,
 183,
 189,
 190,
 191,
 192,
 193,
 197,
 198,
 202,
 203,
 204,
 209,
 220,
 224,
 225,
 226,
 227,
 229,
 230,
 231,
 259,
 263,
 265,
 266,
 268,
 276,
 277,
 278,
 279,
 280,
 288,
 289,
 297,
 298,
 299,
 300,
 307,
 321,
 322,
 327,
 340,
 346,
 360,
 362,
 363,
 364,
 365,
 366,
 367,
 369,
 371,
 372,
 373,
 374,
 375,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 405,
 406,
 407,
 408,
 414,
 415,
 416,
 417,
 419,
 428,
 430,
 433,
 434,
 437,
 463,
 478,
 479,
 480,
 481,
 482,
 483,
 487,
 492,
 495,
 496,
 505,
 518,
 527,
 539,
 540,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 550,
 551,
 552,
 553,
 554,
 555,
 556,
 558,
 559,
 560,
 563,
 575,
 576,
 577,
 578,
 580,
 581,
 582,
 584,
 585,
 586,
 588,
 591,
 592,
 593,
 594,
 612,
 613,
 618